# [Naive Bayes 單純貝氏定理](https://scikit-learn.org/stable/modules/naive_bayes.html)

- 單純貝氏（Naive Bayes）屬於**分類型問題（classification）**
- 公式：$ \small P(A) \times P \large(\frac{B}{A}) \small = P(B) \times P \large(\frac{A}{B}) \implies \small P \large(\frac{B}{A})\small = \large \frac{P(B) \times P(A|B)}{P(A)} $

- $ \small P(A) $ ：事件 A 發生的機率
- $ \small P(A|B) $ or $ \small P(\large \frac{A}{B} \small) $ ：B 條件下，A 發生的機率
- $ P(A,B) $ ： A,B 同時發生的機率
  - **非獨立事件**：$ \small P(A,B) = P(A) \times P \large(\frac{B}{A}) \small = P(B) \times P \large(\frac{A}{B}) $    
    例子：今天下雨，而明天再下雨的機率
  - **獨　立事件**：$ \small P(A,B) = P(A) \times P(B) $    
    例子：二人各自擲骰子的機率

-------------------------------------------------------------------------------------- 
- 遇到**文字型資料**時，無法使用決策樹做機器學習，因為**欄位太多**
- 欄位太多會讓資料分布很稀疏，會引發「**維度災難 Curse of Dimensionality**」
- 欄位太多會造成 (1)演算法無法負荷、(2)需要大量資料才能 cover 更多的欄位
- 因此，透過「**單純貝氏定理**」可以解決維度災難的問題。
  - 單純貝氏定理，又稱錯誤貝氏，因為假設**獨立事件**
  - 而因假設**獨立事件**，故不會出現**維度組合**的狀況
  - 因此不需要大量資料，而演算法也可計算得出

---------------------------------------------------------------------------------------
### 【簡化此題】  
**題目：預測詩詞「我 喜歡 你」的作詩者為何人。  
目標：機率比大小，機率大者則推斷此人為作詩者。**  

$ \small P \large(\frac{\text{李白}}{\text{我喜歡你}})\small = \large \frac{P(\text{李白}) \times P(\text{我喜歡你|李白})}{P(\text{我喜歡你})} $  

P(李白|我喜歡你) =（P(李白) × P(我喜歡你|李白)）÷ P(我喜歡你)   
   - P(李白) = 任一首詩是李白寫的機率 =（李白的詩(train) ÷ 所有的詩(train)）   
   - P(我喜歡你|李白) = 李白寫出我喜歡你這首詩的機率  
   - P(我喜歡你) = 任一詩人寫出我喜歡你這首詩的機率     

- 因只是要比較機率大小，而 **分母值 = P(我喜歡你)** 難以計算，    
  但大家都是除相同的分母，故可直接忽略不考慮分母沒關係XD
  
獨　立事件：P(我喜歡你|李白) = P(我) × P(喜歡) × P(你) = P(我|李白) × P(喜歡|李白) × P(你|李白)   
非獨立事件：P(我喜歡你|李白) = P(我) × P(喜歡|我) × P(你|我喜歡) 

  
- 因只是要比較機率大小，故可用**獨立事件**的方式計算，  
  雖然此計算的機率會失準、有誤差，但大家在相同條件下，所以沒關係XD  
  若是用**非獨立事件**的方式計算，演算法算不出來QQ 

---------------------------------------------------------------------------------------
- [Multinomial Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes)：  
  - 最常使用，在文字型資料的模型訓練時
  - 使用時機：特徵是固定值或分類型，如擲骰子只會有1~6
  - 計算方法：透過公式來計算機率值
  - 公式：$ \large \hat{\theta}_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha_n} $  
    $ N_{y} =$ 某人總用詞數   
    $ N_{yi} =$ 某人用某字的次數   
    $ \alpha_n =$ 總欄位   
    $ \alpha   = 1 $    
  - smoothing 平滑化：  
    為了避免機率出現「零」而影響預測，因此設置 Laplace smoothing：$\alpha = 1$  
  
  
- [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)： 
  - 不常用
  - 使用時機：特徵是連續值 
  - 計算方法：透過擬合高斯常態分佈的機率，來計算機率值
  
- [Bernoulli Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#bernoulli-naive-bayes)： 
  - 二值型 MultinomialNB
---------------------------------------------------------------------------------------

## step1: 處理資料

In [1]:
import pandas as pd

train_df = pd.read_csv("poem/poem_train.csv", encoding="utf-8")
test_df  = pd.read_csv("poem/poem_test.csv", encoding="utf-8")
train_df

,作者,詩名,內容
0,李白,菩薩蠻·平林漠漠煙如織,平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r...
1,李白,把酒問月,青天有月來幾時，我今停杯一問之：人攀明月不可得，月行卻與人相隨？皎如飛鏡臨丹闕，綠煙滅儘清輝...
2,李白,春思,燕草如碧絲，秦桑低綠枝。當君懷歸日，是妾斷腸時。春風不相識，何事入羅幃。
3,李白,春夜洛城聞笛,誰家玉笛暗飛聲，散入春風滿洛城。此夜曲中聞折柳，何人不起故園情。
4,李白,古風 其十九,西上蓮花山，迢迢見明星。(西上 一作：西嶽)素手把芙蓉，虛步躡太清。霓裳曳廣帶，飄拂升天行。...
...,...,...,...
2726,白居易,彆元九後詠所懷,零落桐葉雨，蕭條槿花風。悠悠早秋意，生此幽閒中。況與故人彆，中懷正無悰。勿雲不相送，心到青門...
2727,白居易,早秋曲江感懷,離離暑雲散，嫋嫋涼風起。池上秋又來，荷花半成子。朱顏易銷歇，白日無窮已。人壽不如山，年光急於...
2728,白居易,東墟晚歇 時退居渭村。,涼風冷露蕭索天，黃蒿紫菊荒涼田。繞塚秋花少顏色，細蟲小蝶飛翻翻。中有騰騰獨行者，手拄漁竿不騎...
2729,白居易,南秦雪,往歲曾為西邑吏，慣從駱口到南秦。\r\n三時雲冷多飛雪，二月山寒少有春。\r\n我思舊事猶惆...


In [2]:
# 做模型時，傳入值必須是數字，故將目標/答案轉為：0,1,2
trans = {"李白":0, "杜甫":1, "白居易":2}

# 反轉目標/答案字典，之後解讀資料用
reverse_trans = {v:k for k, v in trans.items()}
reverse_trans

{0: '李白', 1: '杜甫', 2: '白居易'}

In [3]:
y_train = train_df["作者"].replace(trans)
y_test  = test_df["作者"].replace(trans)

### 利用 `jieba` 做文字分詞

In [4]:
import jieba

s = "平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。"
s = " ".join(jieba.cut(s))
s = s.replace("\r","").replace("\n","")
s

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yalon\AppData\Local\Temp\jieba.cache
Loading model cost 0.681 seconds.
Prefix dict has been built successfully.


'平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。  暝 色入 高樓 ， 有人 樓上 愁 。 玉階空 佇立 ， 宿鳥 歸 飛急 。'

### 透過 `pandas.Series.apply` 對所有的資料做文字分詞

In [5]:
def poemcut(s):
    s = " ".join(jieba.cut(s))
    s = s.replace("\r","").replace("\n","")
    return s

train = train_df["內容"].apply(poemcut)
test  = test_df["內容"].apply(poemcut)

### 計算特徵資料（每首詩對於每個詞）的出現次數
sklearn.feature_extraction.text.CountVectorizer 

- 訓練資料：  
  - 統計特徵欄位多少種：`fit`   
  - 轉換成特徵出現次數：`transform` （次數是根據特徵欄位統計的）
  - 所以使用 `fit_transform(self, raw_documents[, y])`
    

- 測試資料：  
  - 不可以做 `fit`！只要遇到沒看過的欄位資料（特徵），就直接刪除  
  - 所以只能 `transform(self, raw_documents)`


### TfidfVectorizer
- 事先計算出特徵重要性（不需要篩選特徵）
- 字詞在文章中的重要程度 = Tfidf = 字詞出現次數 × (1 ÷ 字詞常用度) 
- Tfidf 的值介於 0.01 到 0.1 之間 ，因此 Tfidf 的 $\alpha$ 要調整為最小值  $\alpha = 0.01$

In [6]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vec = TfidfVectorizer()
# x_train = vec.fit_transform(train)
# x_test  = vec.transform(test)

### CountVectorizer

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
x_train = vec.fit_transform(train)
x_test  = vec.transform(test)

In [8]:
# 注意！標點符號有沒有被去掉，KeyError表示已經去掉了～
# vec.vocabulary_["。"]

In [9]:
# 每個欄位對應到的詞彙，是個字典形式
# print(vec.vocabulary_)

# 反轉特徵欄位字典，之後解讀資料用
reverse_voca = { v:k for k, v in vec.vocabulary_.items()}
reverse_voca[8048]

'受塵'

In [10]:
# 共有 2731(row) x 52294(column) 筆資料 
# sparse matrix 稀疏矩陣，表示只儲存非「零」的值
# 故只有 85677 筆資料被存下來

# e.g.:( 0 , 16053 )  1
#   -> (row, column)  data

print(x_train)
x_train

  (0, 16053)	1
  (0, 29006)	1
  (0, 30177)	1
  (0, 14002)	1
  (0, 139)	1
  (0, 4756)	1
  (0, 39345)	1
  (0, 51400)	1
  (0, 23289)	1
  (0, 25180)	1
  (0, 31382)	1
  (0, 3549)	1
  (0, 13846)	1
  (0, 50568)	1
  (0, 3797)	1
  (0, 26005)	1
  (0, 44987)	2
  (0, 34)	1
  (0, 33975)	1
  (1, 49338)	1
  (1, 16293)	1
  (1, 19165)	1
  (1, 4540)	1
  (1, 85)	1
  (1, 2776)	1
  :	:
  (2729, 8800)	1
  (2729, 6485)	1
  (2729, 3060)	1
  (2730, 25782)	1
  (2730, 6026)	1
  (2730, 36958)	1
  (2730, 15905)	1
  (2730, 44854)	1
  (2730, 29458)	1
  (2730, 2175)	1
  (2730, 31125)	1
  (2730, 40974)	1
  (2730, 35446)	1
  (2730, 49196)	1
  (2730, 6770)	1
  (2730, 48596)	1
  (2730, 7813)	1
  (2730, 1714)	1
  (2730, 39190)	1
  (2730, 28820)	1
  (2730, 38489)	1
  (2730, 28423)	1
  (2730, 51689)	1
  (2730, 8048)	1
  (2730, 23226)	1


<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

## step2: 訓練模型

In [11]:
from sklearn.naive_bayes import MultinomialNB

# CountVectorizer
clf = MultinomialNB() 
# TfidfVectorizer
# clf = MultinomialNB(alpha=0.0001)
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## step3: 預測結果

In [12]:
pre = clf.predict(x_test)
print('預測結果：', list(pre))
print('正確結果：', list(y_test))

預測結果： [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確結果： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


## step4: 驗證模型

In [13]:
from sklearn.metrics import accuracy_score
score = accuracy_score(pre, y_test)
print('正確率：', score)

正確率： 0.8


In [14]:
from sklearn.metrics import confusion_matrix

print('分類準確性評估：')
mat = confusion_matrix(y_test, pre)

ori = ["李白","杜甫","白居易"]
c = ["{}(預測)".format(s) for s in ori]
r = ["{}(目標)".format(s) for s in ori]

pd.DataFrame(mat, columns=c, index=r)

分類準確性評估：


,李白(預測),杜甫(預測),白居易(預測)
李白(目標),8,1,1
杜甫(目標),1,8,1
白居易(目標),1,1,8


## 預測APP：讓使用者輸入詩句做預測

In [15]:
s = input("請輸入一首詩：") 
s = [poemcut(s)]           # 做文字分詞，並以 list or pd.Series 方式傳入（必須）
s = vec.transform(s)       # 將資料轉換成特徵出現次數
pre = clf.predict(s)[0]    # 預測結果，是一個 array，故要透過 [0] 取出第一筆預測結果
ans = reverse_trans[pre]   # 將預測的結果，由數字轉為文字
print("作詩人應該是：", ans)

請輸入一首詩：平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。
作詩人應該是： 李白
